IR pr 5

Implement Page Rank Algorithm. (Use python or beautiful soup for implementation).

In [1]:
import networkx as nx

def pagerank(G, alpha=0.85, personalization=None, 
             max_iter=100, tol=1.0e-6, weight='weight', 
             dangling=None, nstart=None):

    if len(G) == 0:
        return {}
  
    # Make sure graph is directed
    if not G.is_directed():
        D = G.to_directed()
    else:
        D = G
        
    # Convert to right stochastic form
    W = nx.stochastic_graph(D, weight=weight)
    N = W.number_of_nodes()
    
    # Choose starting vector if not given
    if nstart is None:
        x = dict.fromkeys(W, 1.0 / N)
    else:
        s = float(sum(nstart.values()))
        x = dict((k, v / s) for k, v in nstart.items())
        
    # Assign uniform personalization vector if not given 
    if personalization is None:
        p = dict.fromkeys(W, 1.0 / N)
    else:
        missing = set(G) - set(personalization)
        if missing:
            raise nx.NetworkXError("Personalization dictionary must have a value for every node")
        s = float(sum(personalization.values()))
        p = dict((k, v / s) for k, v in personalization.items())
        
    # Handle dangling nodes
    if dangling is None:
        dangling_weights = p
    else:
        missing = set(G) - set(dangling)
        if missing:
            raise nx.NetworkXError("Dangling node dictionary must have a value for every node")
        s = float(sum(dangling.values()))
        dangling_weights = dict((k, v / s) for k, v in dangling.items())
        
    dangling_nodes = [n for n in W if W.out_degree(n, weight=weight) == 0.0]
    
    # Power iteration: iterate until convergence
    for _ in range(max_iter):
        xlast = x
        x = dict.fromkeys(xlast.keys(), 0)
        danglesum = alpha * sum(xlast[n] for n in dangling_nodes)
        
        for n in xlast:
            for nbr in W[n]:
                x[nbr] += alpha * xlast[n] * W[n][nbr][weight]
        for n in x:
            x[n] += danglesum * dangling_weights[n] + (1.0 - alpha) * p[n]
  
        # Check convergence
        err = sum(abs(x[n] - xlast[n]) for n in x)
        if err < N * tol:
            return x

    raise nx.NetworkXError("Pagerank: power iteration failed to converge")


In [2]:
import networkx as nx

# Create a random Barabási–Albert graph
G = nx.barabasi_albert_graph(60, 41) 

# Compute PageRank using NetworkX’s built-in function
pr = nx.pagerank(G, alpha=0.4)

print(pr)


{0: 0.028140754356538296, 1: 0.01357121312059036, 2: 0.012364817594794228, 3: 0.012947394757077627, 4: 0.012579602095048716, 5: 0.013359273804006977, 6: 0.012563181397984126, 7: 0.013361023387481945, 8: 0.012954321686913451, 9: 0.012761511380706487, 10: 0.012743205133155102, 11: 0.013777822708937265, 12: 0.013189390633081333, 13: 0.013367693877617246, 14: 0.013165119415822713, 15: 0.012553868765815676, 16: 0.012971181359508801, 17: 0.012767041193923972, 18: 0.012171196605245443, 19: 0.012956528445964847, 20: 0.013376099455234458, 21: 0.01338072253901845, 22: 0.012970423672175957, 23: 0.01276638958778285, 24: 0.012964677584817496, 25: 0.013581017190426991, 26: 0.013777822708937265, 27: 0.012383392086373365, 28: 0.013582208698154427, 29: 0.012766694341643053, 30: 0.013152664215660073, 31: 0.012548794032799784, 32: 0.012581343542569118, 33: 0.012965366742511707, 34: 0.012354382770135987, 35: 0.011586199691226588, 36: 0.012774065834191774, 37: 0.012569282602096098, 38: 0.013152833619048183